# WandbCallbackHandler Demo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
load_dotenv("/Users/ayushthakur/integrations/llamaindex/apis.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from llama_index.callbacks import CallbackManager, CBEventType
from llama_index.callbacks import LlamaDebugHandler, WandbCallbackHandler
from llama_index import (
    GPTListIndex, GPTTreeIndex, GPTVectorStoreIndex,
    ServiceContext, SimpleDirectoryReader, LLMPredictor,
    GPTSimpleKeywordTableIndex, StorageContext
)
# from llama_index.indices.composability import ComposableGraph
from llama_index import load_index_from_storage, load_graph_from_storage
from langchain.chat_models import ChatOpenAI

# Setup LLM

In [6]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0))

# W&B Callback Manager Setup

In [5]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)

# wandb.init args
run_args = dict(
    project="llamaindex-demo",
)

wandb_callback = WandbCallbackHandler(run_args=run_args)

callback_manager = CallbackManager([llama_debug, wandb_callback])
# callback_manager = CallbackManager([llama_debug])
service_context = ServiceContext.from_defaults(callback_manager=callback_manager, llm_predictor=llm_predictor)

wandb: Streaming LlamaIndex events to W&B at https://wandb.ai/ayut/llamaindex-demo/runs/p2exfawz
wandb: `WandbCallbackHandler` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `llamaindex`.


> After running the above cell, you will get the W&B run page URL. Here you will find a trace table with all the events tracked using [Weights and Biases' Prompts](https://docs.wandb.ai/guides/prompts) feature.

# 1. Indexing

In [21]:
docs = SimpleDirectoryReader("../data/paul_graham/").load_data()

In [22]:
index = GPTVectorStoreIndex.from_documents(docs, service_context=service_context)

**********
Trace: index_construction
    |_node_parsing ->  0.106265 seconds
      |_chunking ->  0.105889 seconds
    |_embedding ->  1.728896 seconds
    |_embedding ->  1.306358 seconds
**********


wandb: WARNING Serializing object of type str that is 150172 bytes
wandb: WARNING Serializing object of type str that is 153936 bytes
wandb: WARNING Serializing object of type str that is 150172 bytes
wandb: WARNING Serializing object of type str that is 153936 bytes
wandb: WARNING Serializing object of type str that is 150112 bytes
wandb: Logged trace tree to W&B.


## 1.1 Persist Index as W&B Artifacts

In [47]:
wandb_callback.upload_index(index, index_name="simple_vector_store")

wandb: Adding directory to artifact (/Users/ayushthakur/integrations/llamaindex/llama_index/docs/examples/callbacks/wandb/run-20230602_193417-vslwisv6/files/storage)... Done. 0.0s


## 1.2 Download Index from W&B Artifacts

In [48]:
storage_context, _ = wandb_callback.download_index(artifact_url="ayut/llamaindex-demo/simple_vector_store:v2")

# Load the index and initialize a query engine
index = load_index_from_storage(storage_context, service_context=service_context)

wandb:   3 of 3 files downloaded.  


**********
Trace: index_construction
**********


# 2. Query Over Index

In [24]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response, sep="\n")

**********
Trace: query
    |_query ->  5.457073 seconds
      |_retrieve ->  0.554004 seconds
        |_embedding ->  0.537537 seconds
      |_synthesize ->  4.902839 seconds
        |_llm ->  4.870042 seconds
**********


wandb: Logged trace tree to W&B.


The context information does not provide information about the author's upbringing or childhood. It mainly discusses the author's experiences and activities as an adult, including writing essays, working on various projects, and meeting his future wife.


# 3. Build Complex Indices

In [50]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [51]:
# load NYC dataset
nyc_documents = SimpleDirectoryReader('data/').load_data()
# load PG's essay
essay_documents = SimpleDirectoryReader('../paul_graham_essay/data/').load_data()

In [52]:
# While building a composable index, to correctly save the index,
# the same `storage_context` needs to be passed to every index.
storage_context = StorageContext.from_defaults()

In [53]:
# build NYC index
nyc_index = GPTVectorStoreIndex.from_documents(
    nyc_documents, service_context=service_context, storage_context=storage_context
)

**********
Trace: index_construction
    |_node_parsing ->  0.170667 seconds
      |_chunking ->  0.170026 seconds
    |_embedding ->  1.655455 seconds
    |_embedding ->  1.119139 seconds
    |_embedding ->  1.122665 seconds
    |_embedding ->  0.850398 seconds
**********


wandb: WARNING Serializing object of type str that is 261554 bytes
wandb: WARNING Serializing object of type str that is 268334 bytes
wandb: WARNING Serializing object of type str that is 261554 bytes
wandb: WARNING Serializing object of type str that is 268334 bytes
wandb: WARNING Serializing object of type str that is 261494 bytes
wandb: Logged trace tree to W&B.


In [54]:
# build essay index
essay_index = GPTVectorStoreIndex.from_documents(
    essay_documents, service_context=service_context, storage_context=storage_context
)

**********
Trace: index_construction
    |_node_parsing ->  2.3e-05 seconds
**********


wandb: Logged trace tree to W&B.


## 3.1. Query Over Graph Index

In [55]:
nyc_index_summary = """
    New York, often called New York City or NYC, 
    is the most populous city in the United States. 
    With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), 
    New York City is also the most densely populated major city in the United States, 
    and is more than twice as populous as second-place Los Angeles. 
    New York City lies at the southern tip of New York State, and 
    constitutes the geographical and demographic center of both the 
    Northeast megalopolis and the New York metropolitan area, the 
    largest metropolitan area in the world by urban landmass.[8] With over 
    20.1 million people in its metropolitan statistical area and 23.5 million 
    in its combined statistical area as of 2020, New York is one of the world's 
    most populous megacities, and over 58 million people live within 250 mi (400 km) of 
    the city. New York City is a global cultural, financial, and media center with 
    a significant influence on commerce, health care and life sciences, entertainment, 
    research, technology, education, politics, tourism, dining, art, fashion, and sports. 
    Home to the headquarters of the United Nations, 
    New York is an important center for international diplomacy,
    an established safe haven for global investors, and is sometimes described as the capital of the world.
"""
essay_index_summary = """
    Author: Paul Graham. 
    The author grew up painting and writing essays. 
    He wrote a book on Lisp and did freelance Lisp hacking work to support himself. 
    He also became the de facto studio assistant for Idelle Weber, an early photorealist painter. 
    He eventually had the idea to start a company to put art galleries online, but the idea was unsuccessful. 
    He then had the idea to write software to build online stores, which became the basis for his successful company, Viaweb. 
    After Viaweb was acquired by Yahoo!, the author returned to painting and started writing essays online. 
    He wrote a book of essays, Hackers & Painters, and worked on spam filters. 
    He also bought a building in Cambridge to use as an office. 
    He then had the idea to start Y Combinator, an investment firm that would 
    make a larger number of smaller investments and help founders remain as CEO. 
    He and his partner Jessica Livingston ran Y Combinator and funded a batch of startups twice a year. 
    He also continued to write essays, cook for groups of friends, and explore the concept of invented vs discovered in software. 
"""

In [56]:
from llama_index import StorageContext, load_graph_from_storage

graph = ComposableGraph.from_indices(
    GPTSimpleKeywordTableIndex,
    [nyc_index, essay_index], 
    index_summaries=[nyc_index_summary, essay_index_summary],
    max_keywords_per_chunk=50,
    service_context=service_context,
    storage_context=storage_context
)

**********
Trace: graph_construction
**********


### 3.1.1 Persist Composable Index as W&B Artifacts 

In [39]:
wandb_callback.upload_index(graph, index_name="composable_graph")

wandb: Adding directory to artifact (/Users/ayushthakur/integrations/llamaindex/llama_index/docs/examples/callbacks/wandb/run-20230602_192905-yo6en1x2/files/storage)... Done. 0.0s


### 3.1.2 Download Index from W&B Artifacts

In [40]:
storage_context, _ = wandb_callback.download_index(artifact_url="ayut/llamaindex-demo/composable_graph:v1")

# Load the graph and initialize a query engine
graph = load_graph_from_storage(storage_context, root_id=graph.root_id, service_context=service_context)
query_engine = index.as_query_engine()

wandb:   3 of 3 files downloaded.  


**********
Trace: index_construction
**********
**********
Trace: index_construction
**********
**********
Trace: index_construction
**********
**********
Trace: index_construction
**********
**********
Trace: index_construction
**********
**********
Trace: index_construction
**********
**********
Trace: index_construction
**********


### 3.1.3 Query

In [57]:
response = query_engine.query(
    "What is the climate of New York City like? How cold is it during the winter?", 
)

**********
Trace: query
    |_query ->  3.225175 seconds
      |_retrieve ->  0.333519 seconds
        |_embedding ->  0.328494 seconds
      |_synthesize ->  2.891545 seconds
        |_llm ->  2.872739 seconds
**********


wandb: Logged trace tree to W&B.


### Close W&B Callback Handler

When we are done tracking our events we can close the wandb run.

In [25]:
wandb_callback.finish()